In [ ]:
import cv2
import numpy as np
import tensorflow as tf
tflite = tf.lite
from math import atan2, degrees

# -------------------------------
# Utility functions
# -------------------------------
def draw_keypoints(frame, keypoints, threshold=0.3):
    h, w, _ = frame.shape
    for y, x, score in keypoints:
        if score > threshold:
            cv2.circle(frame, (int(x * w), int(y * h)), 5, (0, 255, 0), -1)

def draw_connections(frame, keypoints, edges, threshold=0.3):
    h, w, _ = frame.shape
    for (start, end) in edges:
        y1, x1, s1 = keypoints[start]
        y2, x2, s2 = keypoints[end]
        if s1 > threshold and s2 > threshold:
            cv2.line(frame, (int(x1*w), int(y1*h)), (int(x2*w), int(y2*h)), (0,255,255), 2)

def calculate_angle(a, b):
    """Calculate angle between two points (a,b) in degrees relative to vertical"""
    dy = a[1] - b[1]
    dx = a[0] - b[0]
    return degrees(atan2(dy, dx))

def midpoint(a, b):
    return ((a[0]+b[0])/2, (a[1]+b[1])/2)

def draw_line(frame, pt1, pt2, color=(255,0,0), thickness=2, label=""):
    h, w, _ = frame.shape
    pt1_pixel = (int(pt1[0]*w), int(pt1[1]*h))
    pt2_pixel = (int(pt2[0]*w), int(pt2[1]*h))
    cv2.line(frame, pt1_pixel, pt2_pixel, color, thickness)
    if label:
        cv2.putText(frame, label, (pt2_pixel[0]+5, pt2_pixel[1]+5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# -------------------------------
# Load TFLite MoveNet model
# -------------------------------
interpreter = tflite.Interpreter(model_path="movenet-tflite-singlepose-lightning-v1.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# -------------------------------
# Skeleton edges
# -------------------------------
EDGES = [
    (0, 1), (0, 2), (1, 3), (2, 4),
    (0, 5), (0, 6),
    (5, 7), (7, 9),
    (6, 8), (8,10),
    (5,11), (6,12),
    (11,12),
    (11,13), (13,15),
    (12,14), (14,16)
]

# -------------------------------
# Video capture
# -------------------------------
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (192,192))
    input_image = np.expand_dims(img_resized.astype(np.float32), axis=0)

    # Run inference
    interpreter.set_tensor(input_details[0]['index'], input_image)
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])[0][0]

    draw_connections(frame, keypoints_with_scores, EDGES)
    draw_keypoints(frame, keypoints_with_scores)

    # -------------------------------
    # Posture detection
    # -------------------------------
    nose = keypoints_with_scores[0][:2]
    left_shoulder = keypoints_with_scores[5][:2]
    right_shoulder = keypoints_with_scores[6][:2]
    left_hip = keypoints_with_scores[11][:2]
    right_hip = keypoints_with_scores[12][:2]

    shoulder_mid = midpoint(left_shoulder, right_shoulder)
    hip_mid = midpoint(left_hip, right_hip)

    torso_angle = calculate_angle(hip_mid, shoulder_mid)
    neck_angle = calculate_angle(shoulder_mid, nose)

    # -------------------------------
    # Draw angle lines
    draw_line(frame, shoulder_mid, nose, color=(0,0,255), label=f"Neck {neck_angle:.1f} deg")
    draw_line(frame, hip_mid, shoulder_mid, color=(255,0,0), label=f"Torso {torso_angle:.1f} deg")

    # -------------------------------
    # Good/Bad posture thresholds
    # (You can adjust these)
    torso_good = abs(torso_angle) < 10       # near vertical
    neck_good = abs(neck_angle) < 20         # slight forward tilt allowed

    posture_text = "GOOD" if torso_good and neck_good else "BAD"
    color = (0,255,0) if posture_text=="GOOD" else (0,0,255)
    cv2.putText(frame, f"Posture: {posture_text}", (10,90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # -------------------------------
    cv2.imshow("Posture Tracking", frame)
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
